#### Script to Predict the output on new observations

In [1]:
import pandas as pd
import pickle
import numpy as np

In [12]:
test_data = pd.read_csv("test_data_subset.csv")
test_data.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In [3]:
#load the columns to drop file
columns_to_drop=pd.read_csv("columns_to_drop.csv")
columns_to_Retain = set(test_data.columns.values) - set(columns_to_drop.colnames.values)
test_data = test_data[columns_to_Retain]

column_datatypes = test_data.dtypes
categorical_columns = list(column_datatypes[column_datatypes=="object"].index.values)

In [4]:
#Transfor the categorical columns by loading the fit encodings on training data

for cf1 in categorical_columns:
    filename = cf1+".sav"
    le = pickle.load(open(filename, 'rb'))
    
    #if an new classes is observed, set it to the 0 class
    le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
    test_data[cf1]=test_data[cf1].apply(lambda x: le_dict.get(x, -1))

In [5]:
test_data_id = test_data['id']
test_data = test_data.drop('id',axis=1)

In [6]:
Column_datatypes= test_data.dtypes
Integer_columns = list(Column_datatypes.where(lambda x: x =="int64").dropna().index.values)
#convert the int64 columns categorical
test_data[Integer_columns] = test_data[Integer_columns].astype('category',copy=False)

In [7]:
#load the saved model and predict on the test data
tunedmodel_rf = pickle.load(open("tunedmodel_rf", 'rb'))

In [8]:
Y_test_predict = tunedmodel_rf.predict(test_data)
test_data['predict_loss']=Y_test_predict
test_data['id']=test_data_id
test_data = test_data[['id','predict_loss']]

In [9]:
test_data

,id,predict_loss
0,4,8.156381
1,6,7.988558
2,9,8.033879
3,12,8.277218
4,15,7.823140
